In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/learning-agency-lab-automated-essay-scoring-2/sample_submission.csv
/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv
/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv


import other libraries

In [2]:
!pip install mord

  Preparing metadata (setup.py) ... - done
  Created wheel for mord: filename=mord-0.7-py3-none-any.whl size=9886 sha256=b03cd541c3170f9d958029ab526891278e66e5d94875072e81c74372c2005a40
  Stored in directory: /root/.cache/pip/wheels/77/00/19/3cea86fbfc737ec4acb515cd94497dcc33f943fa157548b96c
Successfully built mord


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import cohen_kappa_score, make_scorer, classification_report, accuracy_score, confusion_matrix
import re
from sklearn.pipeline import Pipeline
# from sklearn.pipeline import FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV, train_test_split, cross_validate
from sklearn.preprocessing import StandardScaler, FunctionTransformer

from gensim.models import Word2Vec
from mord import OrdinalRidge
import seaborn as sns
# import statsmodels.api as sm
# from statsmodels.miscmodels.ordinal_model import OrderedModel

Read in the dataset

In [4]:
df_train = pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv')
df_test = pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv')
df_sub = pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/sample_submission.csv')

Data cleaning

In [5]:
def removeHTML(x):
    html=re.compile(r'<.*?>')
    return html.sub(r'',x)

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r"\xa0"," ",phrase)
    return phrase.strip()

def clean_text(x):
    # Convert words to lowercase
    x = x.lower()
    # Remove HTML
    x = removeHTML(x)
    # Delete strings starting with @
    x = re.sub("@\w+", '',x)
    # Delete Numbers
    x = re.sub("'\d+", '',x)
    x = re.sub("\d+", '',x)
    # Delete URL
    x = re.sub("http\w+", '',x)
    # Replace consecutive empty spaces with a single space character
    x = re.sub(r"\s+", " ", x)
    # Replace consecutive commas and periods with one comma and period character
    x = re.sub(r"\.+", ".", x)
    x = re.sub(r"\,+", ",", x)
    # Remove empty characters at the beginning and end
    x = x.strip()
    return decontracted(x)

In [6]:
df_train['full_text'] = df_train['full_text'].apply(clean_text)
df_test['full_text'] = df_test['full_text'].apply(clean_text)


In [7]:
df_train['full_text'][0]

'many people have car where they live. the thing they do not know is that when you use a car alot of thing can happen like you can get in accidet or the smoke that the car has is bad to breath on if someone is walk but in vauban,germany they dont have that proble because percent of vauban is families do not own cars,and percent sold a car to move there. street parkig ,driveways and home garages are forbidden on the outskirts of freiburd that near the french and swiss borders. you probaly will not see a car in vauban is streets because they are completely "car free" but if some that lives in vauban that owns a car ownership is allowed,but there are only two places that you can park a large garages at the edge of the development,where a car owner buys a space but it not cheap to buy one they sell the space for you car for $, along with a home. the vauban people completed this in ,they said that this an example of a growing trend in europe,the untile states and some where else are suburba

feature extraction - maybe add in more features like bag of words/ keywords?

create a word_count and text length column

In [8]:
df_train['word_count'] = df_train["full_text"].str.split().str.len()
df_test['word_count'] = df_test["full_text"].str.split().str.len()

In [9]:
df_train['length'] = df_train["full_text"].str.len()
df_test['length'] = df_test["full_text"].str.len()

In [10]:
df_train.head()

,essay_id,full_text,score,word_count,length
0,000d118,many people have car where they live. the thin...,3,496,2647
1,000fe60,i am a scientist at nasa that is discussing th...,3,336,1668
2,001ab80,people always wish they had the same technolog...,4,553,3068
3,001bdc0,"we all heard about venus, the planet without a...",4,450,2679
4,002ba53,"dear, state senator this is a letter to argue ...",3,377,2188


attempt at creating vector embeddings:

In [11]:
word2vec_model = Word2Vec(df_train['full_text'], 
                          vector_size=100,
                          min_count=1,
                          window=5, 
                          )

In [12]:
def vectorize_text(text, word2vec_model):
    text_vector = np.zeros(100)  # Initialize a vector of zeros (100-dimensional)
    for word in text.split():  # Split text into words
        if word in word2vec_model.wv:  # Check if word is in the vocabulary
            text_vector += word2vec_model.wv[word]  # Add word vector to the text vector
    return text_vector

In [13]:
# Apply the vectorize_text function to each text entry in train_df['full_text']
df_train['text_vector'] = df_train['full_text'].apply(lambda x: vectorize_text(x, word2vec_model))
df_test['text_vector'] = df_test['full_text'].apply(lambda x: vectorize_text(x, word2vec_model))


In [14]:
df_train.head()

,essay_id,full_text,score,word_count,length,text_vector
0,000d118,many people have car where they live. the thin...,3,496,2647,"[-3.4887126088142395, -2.754158705472946, -3.5..."
1,000fe60,i am a scientist at nasa that is discussing th...,3,336,1668,"[-2.581797420978546, -3.308091878890991, -3.38..."
2,001ab80,people always wish they had the same technolog...,4,553,3068,"[-2.4763074666261673, -2.1251330375671387, -2...."
3,001bdc0,"we all heard about venus, the planet without a...",4,450,2679,"[-4.147961877286434, -0.5384019613265991, -1.9..."
4,002ba53,"dear, state senator this is a letter to argue ...",3,377,2188,"[-3.148106172680855, -4.939776241779327, -5.34..."


In [15]:
# X_train = df_train['full_text']
# y_train = df_train['score']
# X_test = df_test['full_text']
# y_test = df_sub['score']

this method trains and evaluates each pipeline 
returns the y_pred and quadratic weighted kappa score of the model

In [16]:
def train_and_evaluate_pipeline(pipeline, X_train, y_train, X_test, y_test):
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    
    kappa = cohen_kappa_score(y_test, y_pred, weights='quadratic')
    
    print()
    print('Classifier used: ', pipeline.named_steps['classifier'])
    print('y_test: ', y_test.tolist()[0:10])
    print('y_pred: ', y_pred.tolist()[0:10])
    print('quadratic weighted kappa score: ', kappa)
    
    return pipeline, y_pred

method for performing cross_validation of pipelines and showing their average performances

In [17]:
#define quadratic weighted kappa as a custom scorer
qwk_scorer = make_scorer(cohen_kappa_score, weights='quadratic')

def pipeline_cross_validation(pipeline, X_train, y_train, qwk_scorer=qwk_scorer):
    # Define cross-validation strategy and perform cross-validation
    scoring = {'accuracy': 'accuracy', 'f1_weighted': 'f1_weighted', 'qwk': qwk_scorer}
    cv_results = cross_validate(pipeline, X_train, y_train, cv=5, scoring=scoring)

    # Print cross-validation results
    print("Mean Accuracy:", np.mean(cv_results['test_accuracy']))
    print("Mean Weighted F1-score:", np.mean(cv_results['test_f1_weighted']))
    print("Mean Quadratic Weighted Kappa Score:", np.mean(cv_results['test_qwk']))

create the tfidf

In [18]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', min_df=0.05, max_df=0.95,)
# tfidf = tfidf_vectorizer.fit_transform(X_train)


create the multinomial logistic regression classifier

In [19]:
logreg_clf = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=500)

create a pipeline for logistic regression classifier

In [20]:
# Define a transformer to extract 'text_vector' from DataFrame
def get_text_vector(df):
    return np.vstack(df['text_vector'])

In [21]:
tfidf_preprocessor = ColumnTransformer(
    transformers=[
        ('text', tfidf_vectorizer, 'full_text'),
        ('word_count', StandardScaler(), ['word_count']),
        ('length', StandardScaler(), ['length'])
    ],
    remainder='passthrough'
)

w2v_preprocessor = ColumnTransformer(
    transformers=[
#         ('text', tfidf_vectorizer, 'full_text'),
        ('text_vector', FunctionTransformer(get_text_vector, validate=False), ['text_vector']),
        ('word_count', StandardScaler(), ['word_count']),
        ('length', StandardScaler(), ['length'])
    ],
    remainder='passthrough'
)

In [22]:
logreg_pipeline = Pipeline(steps=[
#     ('tfidf', tfidf_vectorizer),
    ('preprocessor', tfidf_preprocessor),
    ('classifier', logreg_clf)
])

w2v_logreg_pipeline = Pipeline(steps=[
    ('preprocessor', w2v_preprocessor),
    ('classifier', logreg_clf)
])

fit and make a prediction

In [23]:
# df_sub['essay_id'].tolist()
# df_train.loc[(df_train['essay_id'] == '000d118') | (df_train['essay_id'] =='000fe60') | (df_train['essay_id'] =='001ab80')]
# sub_df_data = df_train.loc[(df_train['essay_id'] == '000d118') | (df_train['essay_id'] =='000fe60') | (df_train['essay_id'] =='001ab80')]
# X_test3 = tfidf_vectorizer.transform(df_test['full_text'])

In [24]:
X = df_train[['full_text', 'word_count', 'length']]
# X = df_train['full_text']
y = df_train['score']
sub_X = df_test[['full_text', 'word_count', 'length']]
sub_y = df_sub['score']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8)

X2 = df_train[['word_count', 'length', 'text_vector']]
sub_X2 = df_test[['word_count', 'length', 'text_vector']]

# X2 = np.array(df_train['text_vector'].tolist())
# sub_X2 = np.array(df_test['text_vector'].tolist())

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y, train_size = 0.8)


# trained_logreg_pipeline, logreg_y_pred = train_and_evaluate_pipeline(logreg_pipeline, X_train, y_train, X_test, y_test)

In [25]:
trained_logreg_pipeline, logreg_y_pred = train_and_evaluate_pipeline(logreg_pipeline, X_train, y_train, X_test, y_test)


Classifier used:  LogisticRegression(max_iter=500, multi_class='multinomial')
y_test:  [1, 2, 2, 5, 3, 1, 1, 4, 3, 5]
y_pred:  [1, 2, 2, 5, 3, 1, 1, 5, 3, 5]
quadratic weighted kappa score:  0.7243650148457756


commented off just to make the notebook save and rerun properly, need to use a different clf if plan to keep

In [26]:
# trained_logreg_pipeline2, logreg_y_pred2 = train_and_evaluate_pipeline(w2v_logreg_pipeline, X_train2, y_train2, X_test2, y_test2)

In [27]:
pipeline_cross_validation(logreg_pipeline, X_train, y_train)

Mean Accuracy: 0.590032502708559
Mean Weighted F1-score: 0.5781088364571707
Mean Quadratic Weighted Kappa Score: 0.7187159791702111


In [28]:
print(classification_report(y_test, logreg_y_pred))

              precision    recall  f1-score   support

           1       0.69      0.28      0.40       275
           2       0.61      0.69      0.65       936
           3       0.58      0.66      0.62      1261
           4       0.56      0.53      0.55       747
           5       0.49      0.30      0.37       206
           6       0.43      0.16      0.24        37

    accuracy                           0.58      3462
   macro avg       0.56      0.44      0.47      3462
weighted avg       0.59      0.58      0.57      3462



In [29]:
print("Model prediction on submission essays: ", trained_logreg_pipeline.predict(sub_X).tolist())
print('Submission essays actual classes: ', sub_y.tolist())

Model prediction on submission essays:  [3, 3, 4]
Submission essays actual classes:  [3, 3, 4]


In [30]:
# logregcv_clf = LogisticRegressionCV(multi_class='multinomial', cv=5, solver='lbfgs', max_iter=1000, refit=True, scoring=qwk_scorer)

create a lightgbm model pipeline

lgbm over xgboost for performance and scalability? maybe elaborate more

In [31]:
lgbm_clf = LGBMClassifier(objective='multiclass', num_class=6, metric='multi_logloss', num_leaves=31, learning_rate=0.1, feature_fraction=0.6)

In [32]:
lgbm_pipeline = Pipeline(steps=[
#     ('tfidf', tfidf_vectorizer),   # TF-IDF Vectorizer
    ('preprocessor', tfidf_preprocessor),
    ('classifier', lgbm_clf) # LightGBM Classifier
])

use some GridSearchCV to find the best parameters

In [33]:
# parameter grid used for the grid search:
param_grid = {
    'num_leaves': [31, 50, 100],
    'learning_rate': [0.05, 0.1, 0.2],
    'feature_fraction': [0.6, 0.8, 0.9]
}

In [34]:
# # create and fit the gridsearch
# grid_search = GridSearchCV(estimator=lgbm_clf, param_grid=param_grid, cv=3, scoring=qwk_scorer)
# grid_search.fit(tfidf_vectorizer.fit_transform(X_train), y_train)

# # Print the best parameters and best score
# print("Best Parameters: ", grid_search.best_params_)
# print("Best Score (quadratic weighted kappa score): ", grid_search.best_score_)

In [35]:
trained_lgbm_pipeline, lgbm_y_pred = train_and_evaluate_pipeline(lgbm_pipeline, X_train, y_train, X_test, y_test)

[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.058023 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 113291
[LightGBM] [Info] Number of data points in the train set: 13845, number of used features: 447
[LightGBM] [Info] Start training from score -2.651193
[LightGBM] [Info] Start training from score -1.296350
[LightGBM] [Info] Start training from score -1.014693
[LightGBM] [Info] Start training from score -1.471357
[LightGBM] [Info] Start training from score -2.897112
[LightGBM] [Info] Start training from score -4.756556
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 w

commented off just to make the notebook run faster for now

In [36]:
# pipeline_cross_validation(lgbm_pipeline, X_train, y_train)

In [37]:
print(classification_report(y_test, lgbm_y_pred))

              precision    recall  f1-score   support

           1       0.72      0.32      0.44       275
           2       0.63      0.64      0.64       936
           3       0.59      0.64      0.61      1261
           4       0.54      0.61      0.57       747
           5       0.50      0.38      0.43       206
           6       0.30      0.16      0.21        37

    accuracy                           0.59      3462
   macro avg       0.55      0.46      0.48      3462
weighted avg       0.59      0.59      0.58      3462



In [38]:
# trained_lgbm_pipeline.predict(df_test[['full_text', 'word_count', 'length']]).tolist()
print("Model prediction on submission essays: ", trained_lgbm_pipeline.predict(sub_X).tolist())
print('Submission essays actual classes: ', sub_y.tolist())

[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
Model prediction on submission essays:  [3, 3, 4]
Submission essays actual classes:  [3, 3, 4]


LogisticRegressionCV classifier model and pipeline

In [39]:
# logregcv_clf = LogisticRegressionCV(multi_class='multinomial', cv=3, solver='lbfgs', max_iter=1000, refit=True, scoring=qwk_scorer)

# logregcv_pipeline = Pipeline(steps=[
# #     ('tfidf', tfidf_vectorizer),
#     ('preprocessor', preprocessor),
#     ('classifier', logregcv_clf)
# ])

# logregcv_y_pred = train_and_evaluate_pipeline(logregcv_pipeline, X_train, y_train, X_test, y_test)

ordinal logistic regression classsifier

In [40]:
# train_input = df_train[['word_count', 'length', 'text_vector']]
# np.asarray(train_input['text_vector'])
# train_input['text_vector'] = np.asarray(train_input['text_vector'])
# train_input

In [41]:
(df_train['text_vector'])

0        [-3.4887126088142395, -2.754158705472946, -3.5...
1        [-2.581797420978546, -3.308091878890991, -3.38...
2        [-2.4763074666261673, -2.1251330375671387, -2....
3        [-4.147961877286434, -0.5384019613265991, -1.9...
4        [-3.148106172680855, -4.939776241779327, -5.34...
                               ...                        
17302    [-1.5619572401046753, 0.4407832622528076, 0.85...
17303    [-3.4951523542404175, -1.427449882030487, -2.8...
17304    [-0.505339190363884, -0.6157466173171997, -0.6...
17305    [-1.2312163412570953, -0.16599619388580322, -0...
17306    [-0.7304580509662628, -0.8089405298233032, -0....
Name: text_vector, Length: 17307, dtype: object

In [42]:
ordreg_clf = OrdinalRidge()

In [43]:
ordreg_pipeline = Pipeline(steps=[
    ('preprocessor', tfidf_preprocessor),
    ('classifier', ordreg_clf) # LightGBM Classifier
])

In [44]:
trained_ordreg_pipeline, ordreg_y_pred = train_and_evaluate_pipeline(ordreg_pipeline, X_train, y_train, X_test, y_test)


Classifier used:  OrdinalRidge()
y_test:  [1, 2, 2, 5, 3, 1, 1, 4, 3, 5]
y_pred:  [1.0, 2.0, 2.0, 5.0, 3.0, 1.0, 1.0, 4.0, 3.0, 6.0]
quadratic weighted kappa score:  0.7409992902892064


In [45]:
print(classification_report(y_test, ordreg_y_pred))

              precision    recall  f1-score   support

           1       0.77      0.25      0.38       275
           2       0.59      0.67      0.63       936
           3       0.57      0.69      0.62      1261
           4       0.59      0.48      0.53       747
           5       0.54      0.36      0.43       206
           6       0.38      0.30      0.33        37

    accuracy                           0.58      3462
   macro avg       0.57      0.46      0.49      3462
weighted avg       0.59      0.58      0.57      3462



In [46]:
print("Model prediction on submission essays: ", trained_ordreg_pipeline.predict(sub_X).tolist())
print('Submission essays actual classes: ', sub_y.tolist())

Model prediction on submission essays:  [2.0, 3.0, 4.0]
Submission essays actual classes:  [3, 3, 4]


In [47]:
pipeline_cross_validation(ordreg_pipeline, X_train, y_train)

Mean Accuracy: 0.5865655471289275
Mean Weighted F1-score: 0.574206898372638
Mean Quadratic Weighted Kappa Score: 0.7348215630192853


In [48]:
# # ordinal_clf = sm.OrdinalModel(y_train, tfidf_vectorizer.transform(X_train['full_text']))

# ordered_model = OrderedModel(y_train,
#                        tfidf_vectorizer.fit_transform(X_train['full_text']).toarray(),
#                         distr='probit')

# ordered_res = ordered_model.fit()
# ordered_res.summary()

In [49]:
# predicted_probs = ordered_results.predict(tfidf_vectorizer.fit_transform(X_test['full_text']).toarray())
# predicted_probs

# ***Things to do:***

- put explanations and more markup in notebook

- more EDA, some diagrams and graphs on the dataset 

    ideas: - compare the cross validation qwk scores, group by scores and show word lengths etc

- try word2vec for vector embeddings and compare with tfidf - make w2v more accurate somehow

- find more models - potential: xgboostclassifier, ordinal logistic regression (sm), SGDClassifier? ,

- overall increase the performance of the models they arent predicting the test dataset 100% correctly

